In [10]:
!pip3 install pymystem3 pandas sklearn

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post9-py3-none-any.whl size=2952 sha256=de085da5188e0680130af47d37bf6a7803a4dbec121af8adf834ac3d03747231
  Stored in directory: /Users/ilya/Library/Caches/pip/wheels/ef/63/d1/f1671e1e93b7ef4d35df483f9b2485e6dd21941da9a92296fb
Successfully built sklearn


In [1]:
import pickle
with open('dss.pickle', 'rb') as file:
    data = pickle.load(file)

In [2]:
import re
import pandas as pd
from pymystem3 import Mystem

def clean(text):
    text = re.sub('[^а-яё ]', ' ', str(text).lower())
    text = re.sub(r" +", " ", text).strip()
    return text

In [3]:
texts = list(map(lambda x: clean(x['text']), data))
labels = list(map(lambda x: x['label'], data))

In [4]:
mstm = Mystem()
normalized = [''.join(mstm.lemmatize(t)[:-1]) for t in texts]

In [5]:
import pandas as pd
df = pd.DataFrame()
df['text'] = texts
df['norm'] = normalized
df['label'] = labels

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=42)
valid, test = train_test_split(test, test_size=0.2, random_state=42)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

model_tfidf = TfidfVectorizer(max_features=5000)

train_tfidf = model_tfidf.fit_transform(train['norm'].values)
valid_tfidf = model_tfidf.transform(valid['norm'].values)
test_tfidf = model_tfidf.transform(test['norm'].values)

In [8]:
from sklearn.ensemble import RandomForestClassifier

cls = RandomForestClassifier(random_state=42)
cls.fit(train_tfidf, train['label'].values)

RandomForestClassifier(random_state=42)

In [9]:
from sklearn.metrics import f1_score
predictions = cls.predict(test_tfidf)
f1_score(predictions, test['label'].values, average='weighted')

0.629399514876379